In [92]:
import pandas as pd

# Cargar el modelo

In [93]:
import pickle

In [94]:
with open('../models/ufc_model.pkl', 'rb') as model_file:
    lgbm = pickle.load(model_file)

# Datasets

In [95]:
metrics_media = pd.read_csv("../data/processed/metrics_media.csv", index_col = "Unnamed: 0")
metrics_media.head()

,fighter,kd,str,td,sub,weight_class
0,Alex Caceres,0.0,43.75,0.00,0.0,2
1,Andrei Arlovski,0.0,27.00,0.00,0.0,4
2,Charles Oliveira,0.0,18.50,0.50,0.5,2
3,Clay Guida,0.0,27.50,1.25,0.0,2
4,Darren Elkins,0.0,33.75,1.25,0.5,2


# Funciones

In [96]:
def generar_enfrentamiento(df, peleador_1_id, peleador_2_id):

    peleador_1 = df[df['fighter'] == peleador_1_id].iloc[0]
    peleador_2 = df[df['fighter'] == peleador_2_id].iloc[0]

    # Comprobar si están en la misma categoría de peso
    if peleador_1['weight_class'] != peleador_2['weight_class']:
        raise ValueError("Los peleadores no están en la misma categoría de peso.")

    metrics = {'r_fighter': peleador_1['fighter'],
                        'b_fighter': peleador_2['fighter'],
                        'r_kd': peleador_1['kd'],
                        'b_kd': peleador_2['kd'],
                        'r_str': peleador_1['str'],
                        'b_str': peleador_2['str'],
                        'r_td': peleador_1['td'],
                        'b_td': peleador_2['td'],
                        'r_sub': peleador_1['sub'],
                        'b_sub': peleador_2['sub'],
                        'weight_class': peleador_1['weight_class']}

    df_versus = pd.DataFrame([metrics])
    return df_versus

In [97]:
def predecir_ganador(peleador_1, peleador_2):

    peso_influencia = {4: 1.5,
                       3: 1.25,
                       2: 1.3}

    puntos_peleador_1 = 0
    puntos_peleador_2 = 0

    # Knockdowns (ponderados por categoría de peso)
    peso_1 = peso_influencia.get(peleador_1['weight_class'], 1.0)
    peso_2 = peso_influencia.get(peleador_2['weight_class'], 1.0)
    
    if peleador_1['kd'] * peso_1 > peleador_2['kd'] * peso_2:
        puntos_peleador_1 += 1
    else:
        puntos_peleador_2 += 1

    # Takedowns
    if peleador_1['td'] > peleador_2['td']:
        puntos_peleador_1 += 1
    else:
        puntos_peleador_2 += 1

    # Strikes (ponderados por categoría de peso)
    if peleador_1['str'] * peso_1 > peleador_2['str'] * peso_2:
        puntos_peleador_1 += 1
    else:
        puntos_peleador_2 += 1

    # Submissions
    if peleador_1['sub'] > peleador_2['sub']:
        puntos_peleador_1 += 1
    else:
        puntos_peleador_2 += 1

    # Sistema de puntuaciones final
    if puntos_peleador_1 > puntos_peleador_2:
        return peleador_1['fighter']
    elif puntos_peleador_2 > puntos_peleador_1:
        return peleador_2['fighter']
    else:
        # Si tienen los mismos puntos, se desempata por el número de strikes
        if peleador_1['str'] > peleador_2['str']:
            return peleador_1['fighter']
        elif peleador_2['str'] > peleador_1['str']:
            return peleador_2['fighter']
        else:
            return "Empate"  # Considerar empate si tienen los mismos strikes también.



# Flujo

In [98]:
versus = generar_enfrentamiento(metrics_media, "Alex Caceres", "Nate Diaz")
versus = versus.drop(columns=["r_fighter","b_fighter"])
versus

,r_kd,b_kd,r_str,b_str,r_td,b_td,r_sub,b_sub,weight_class
0,0.0,0.0,43.75,37.0,0.0,0.25,0.0,0.5,2


In [ ]:
def apuesta(peleador_1, peleador_2, modelo, versus):
    ganador = predecir_ganador(peleador_1, peleador_2)
    resultado = float(modelo.predict(versus)[0])
    
    #Calcular el round y el minuto
    round = resultado // 5
    minuto = resultado / 5
    resto = int(minuto)
    parte_decimal = minuto - resto
    if parte_decimal >= 0.5:
        resto += 1

    return f"Ganará {ganador} en el round: {int(round)} en el minuto {resto}"

In [ ]:
peleador_1 = metrics_media.iloc[6]
peleador_2 = metrics_media.iloc[5]
resultado_prediccion = apuesta(peleador_1, peleador_2, lgbm, versus)
print(resultado_prediccion)

Ganará Demian Maia en el round: 2 en el minuto 3
